# Project on Tartu Smart Bike data analysis
## Introduction to Data Science - LTAT.02.002


## Data exploring
**Used data:**  Confidential Tartu Smart Bike rental and locations data (June 2019 - September 2019)

In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
plt.style.use("ggplot")  # Default style

In [2]:
# reading in data from csv-s
data_routes_june = pd.read_csv("..\\data_for_IDS2019_project_team_W17\\bicycle_data\\routes_201906.csv",sep=',')
data_routes_july = pd.read_csv("..\\data_for_IDS2019_project_team_W17\\bicycle_data\\routes_201907.csv",sep=',')
data_routes_august = pd.read_csv("..\\data_for_IDS2019_project_team_W17\\bicycle_data\\routes_201908.csv",sep=',')
data_routes_september = pd.read_csv("..\\data_for_IDS2019_project_team_W17\\bicycle_data\\routes_201909.csv",sep=',')

### Performing the exploration on the routes dataset from June

In [3]:
data_routes_june.dtypes

route_code            int64
cyclenumber           int64
unlockedat           object
unlockedattime       object
lockedat             object
lockedattime         object
startstationname     object
endstationname       object
rfidnumber           object
length              float64
DurationMinutes     float64
CycleType            object
costs                 int64
Membership           object
dtype: object

In [4]:
for column in data_routes_june.columns:
    print(data_routes_june[column].value_counts())
    print()

1560100206883    2
1560103709436    2
1560098082028    2
1560092928595    2
1560325697121    2
1560186500597    2
1560104282400    2
1560079811191    2
1560019239348    2
1560092040307    2
1560275782057    2
1560105841532    2
1560180777492    2
1559991888983    2
1560273041215    2
1560105323387    2
1560198431598    2
1560017348637    2
1560340586346    2
1560245671235    2
1560058276023    2
1560270771546    2
1560189102885    1
1560267703597    1
1560262561020    1
1560145037546    1
1560074676717    1
1560311929962    1
1560156638452    1
1560262016138    1
                ..
1560260667888    1
1560143223708    1
1560254328799    1
1560190980043    1
1560276572104    1
1560340794816    1
1560016398286    1
1560095305679    1
1560186673088    1
1559990069190    1
1560018120647    1
1559985487813    1
1560189571035    1
1560167948249    1
1560172406750    1
1560329435100    1
1560198430717    1
1560174247901    1
1560025781783    1
1560157867987    1
1560089716694    1
156001068437

In [5]:
# Missing values in the datasets are defined as '-- Määramata --' and 'Undetermined'
def missing_values(data):
    total_rows = len(data.index)
    for column in data.columns:
        total_missing = 0
        if type(data[column].loc[0]) == str:
            total_missing += data[column].str.contains('-- Määramata --').sum()
            total_missing += data[column].str.contains('Undetermined').sum()
        if total_missing > 0:
            print ('Number of missing values in column \'' + column + '\': ' + str(total_missing) )

In [6]:
print("JUNE:")
missing_values(data_routes_june)
print("JULY:")
missing_values(data_routes_july)
print("AUGUST:")
missing_values(data_routes_august)
print("SEPTEMBER:")
missing_values(data_routes_september)

JUNE:
Number of missing values in column 'startstationname': 400
Number of missing values in column 'endstationname': 254
JULY:
Number of missing values in column 'startstationname': 306
Number of missing values in column 'endstationname': 261
AUGUST:
Number of missing values in column 'startstationname': 495
Number of missing values in column 'endstationname': 539
SEPTEMBER:
Number of missing values in column 'startstationname': 490
Number of missing values in column 'endstationname': 628


## Data cleaning
### 1. Removing rows with missing and incorrect values in columns 'startstationname' and 'endstationname'

In [7]:
def remove_missing_rows(data):
    # Missing values in the datasets are defined as '-- Määramata --' and 'Undetermined'
    indices = data[(data['startstationname'] == '-- Määramata --')].index
    data.drop(indices, inplace=True)
    indices = data[(data['startstationname'] == 'Undetermined')].index
    data.drop(indices, inplace=True)
    indices = data[(data['endstationname'] == '-- Määramata --')].index
    data.drop(indices, inplace=True)
    indices = data[(data['endstationname'] == 'Undetermined')].index
    data.drop(indices, inplace=True)
    
    # Stations 'Warehouse', 'Töökoda' and 'Delta' are not publically usable
    indices = data[(data['startstationname'] == 'Warehouse')].index
    data.drop(indices, inplace=True)
    indices = data[(data['endstationname'] == 'Warehouse')].index
    data.drop(indices, inplace=True)
    indices = data[(data['startstationname'] == 'Delta')].index
    data.drop(indices, inplace=True)
    indices = data[(data['endstationname'] == 'Delta')].index
    data.drop(indices, inplace=True)
    indices = data[(data['startstationname'] == 'Töökoda')].index
    data.drop(indices, inplace=True)
    indices = data[(data['endstationname'] == 'Töökoda')].index
    data.drop(indices, inplace=True)
    

In [8]:
remove_missing_rows(data_routes_june)
remove_missing_rows(data_routes_july)
remove_missing_rows(data_routes_august)
remove_missing_rows(data_routes_september)

### 2. Removing rides that are less than 100 m.
These are the 'rides' where the bike is undocked and then re-docked right away.

In [9]:
def remove_less_than_100m_rides(data):
    indices = data[(data['length'] < 0.100)].index
    data.drop(indices, inplace=True)

In [10]:
remove_less_than_100m_rides(data_routes_june)
remove_less_than_100m_rides(data_routes_july)
remove_less_than_100m_rides(data_routes_august)
remove_less_than_100m_rides(data_routes_september)

### 3. Correcting spelling differences in Membership column

In [11]:
data_routes_june.Membership.value_counts()

3 month free membership            20049
3-Month Free Membership            15924
Free Membership (with bus card)     9401
1 month free membership             7555
1-month Free Membership             2965
1 Year Membership                      5
1 YEAR MEMBERSHIP                      4
1 mo                                   1
Name: Membership, dtype: int64

In [12]:
def correct_spelling(data):
    data.Membership = data.Membership.replace(to_replace ="3 month free membership", value = "3-Month Free Membership")
    data.Membership = data.Membership.replace(to_replace ="1 DAY MEMBERSHIP", value = "1 Day Membership")
    data.Membership = data.Membership.replace(to_replace ="1 YEAR MEMBERSHIP", value = "1 Year Membership")
    data.Membership = data.Membership.replace(to_replace ="1 WEEK MEMBERSHIP", value = "1 Week Membership")
    data.Membership = data.Membership.replace(to_replace ="1 month free membership", value = "1-month Free Membership")
    data.drop(data[(data.Membership == 'Free M')].index, inplace=True)
    data.drop(data[(data.Membership == '1 mo')].index, inplace=True)

In [13]:
correct_spelling(data_routes_june)
correct_spelling(data_routes_july)
correct_spelling(data_routes_august)
correct_spelling(data_routes_september)

In [14]:
data_routes_june.Membership.value_counts()

3-Month Free Membership            35973
1-month Free Membership            10520
Free Membership (with bus card)     9401
1 Year Membership                      9
Name: Membership, dtype: int64

### 4. Exporting cleaned datasets to csv 

In [15]:
data_routes_june.to_csv(path_or_buf="Cleaned data\\routes_2019_06_clean.csv",index=False)
data_routes_july.to_csv(path_or_buf="Cleaned data\\routes_2019_07_clean.csv",index=False)
data_routes_august.to_csv(path_or_buf="Cleaned data\\routes_2019_08_clean.csv",index=False)
data_routes_september.to_csv(path_or_buf="Cleaned data\\routes_2019_09_clean.csv",index=False)